Basic implementation of MLEnd Code

In [1]:
!pip install librosa

  Using cached librosa-0.9.2-py3-none-any.whl (214 kB)
  Using cached audioread-3.0.0-py3-none-any.whl
  Using cached resampy-0.4.2-py3-none-any.whl (3.1 MB)
  Using cached pooch-1.6.0-py3-none-any.whl (56 kB)
  Using cached soundfile-0.11.0-py2.py3-none-any.whl (23 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [14]:
#required dependencies

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os, sys, re, pickle, glob
import urllib.request
import zipfile

import IPython.display as ipd
from tqdm import tqdm
import librosa

import zipfile

In [3]:
directory_to_extract_to = '/home/jovyan/ML/DATA/DATA_unzip_full/'

for i in range(1,6):
    zip_path = '/home/jovyan/ML/DATA/DATA_ZIP/MLEndLS_' + str(i) + '.zip'
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)

In [15]:
sample_path = '/home/jovyan/ML/DATA/DATA_unzip_full/*.wav'
files = glob.glob(sample_path)
len(files)

2500

In [16]:
MLENDLS_df = pd.read_csv('/home/jovyan/ML/git-code/MLEndLS_final.csv').set_index('file_id') 
MLENDLS_df

,area,spot,in_out,Participant
file_id,,,,
0001.wav,british,street,outdoor,S151
0002.wav,kensington,dinosaur,indoor,S127
0003.wav,campus,square,outdoor,S18
0004.wav,kensington,hintze,indoor,S179
0005.wav,campus,square,outdoor,S176
...,...,...,...,...
2496.wav,westend,trafalgar,outdoor,S151
2497.wav,campus,square,outdoor,S6
2498.wav,westend,national,indoor,S96


In [17]:
n=0
fs = None
x, fs = librosa.load(files[n],sr=fs)
print('This audio signal has', len(x), 'samples')

This audio signal has 379260 samples


In [18]:
def getPitch(x,fs,winLen=0.02):
  #winLen = 0.02 
  p = winLen*fs
  frame_length = int(2**int(p-1).bit_length())
  hop_length = frame_length//2
  f0, voiced_flag, voiced_probs = librosa.pyin(y=x, fmin=80, fmax=450, sr=fs,
                                                 frame_length=frame_length,hop_length=hop_length)
  return f0,voiced_flag

In [19]:
def getXy(files,labels_file, scale_audio=False, onlySingleDigit=False):
  X,y =[],[]
  for file in tqdm(files):
    fileID = file.split('/')[-1]
    file_name = file.split('/')[-1]
    

    
    try:
        
        fs = None # if None, fs would be 22050 #fr = sr = sampling rate
        x, fs = librosa.load(file,sr=fs)
    
        if scale_audio: x = x/np.max(np.abs(x))
        f0, voiced_flag = getPitch(x,fs,winLen=0.02)
        
        yi = labels_file.loc[fileID]['in_out']=='indoor'
        
        power = np.sum(x**2)/len(x)
        pitch_mean = np.nanmean(f0) if np.mean(np.isnan(f0))<1 else 0
        pitch_std  = np.nanstd(f0) if np.mean(np.isnan(f0))<1 else 0
        voiced_fr = np.mean(voiced_flag)

        xi = [power,pitch_mean,pitch_std,voiced_fr]
        X.append(xi)
        y.append(yi)
        
    except:
        print('issue in file index -->' + str(file))
      
    

  return np.array(X),np.array(y)

initial feature extraction : 

from sound data -> power , pitch, pitch std , voiced fr

In [11]:
files[155]

'/home/jovyan/ML/DATA/DATA_unzip_full/0033.wav'

In [20]:
X,y = getXy(files, labels_file=MLENDLS_df, scale_audio=True, onlySingleDigit=True)

  6%|▌         | 155/2500 [02:31<56:55,  1.46s/it] /tmp/ipykernel_81/2596168170.py:14: RuntimeWarning: invalid value encountered in true_divide
  if scale_audio: x = x/np.max(np.abs(x))


issue in file index -->/home/jovyan/ML/DATA/DATA_unzip_full/0033.wav


 33%|███▎      | 826/2500 [13:16<26:28,  1.05it/s]  

issue in file index -->/home/jovyan/ML/DATA/DATA_unzip_full/2206.wav


100%|██████████| 2500/2500 [41:19<00:00,  1.01it/s]  


In [28]:
print('The shape of X is', X.shape) 
print('The shape of y is', y.shape)
print('The labels vector is', y)

The shape of X is (2498, 4)
The shape of y is (2498,)
The labels vector is [False False  True ... False False  True]


In [29]:
print(' The number of indoor recordings is ', np.count_nonzero(y))
print(' The number of outdoor recordings is ', y.size - np.count_nonzero(y))

 The number of indoor recordings is  1181
 The number of outdoor recordings is  1317


In [30]:
df= pd.DataFrame.from_records(X)
df

,0,1,2,3
0,0.025885,249.622739,7.270292,0.018893
1,0.010622,154.681365,41.506471,0.087582
2,0.033283,0.000000,0.000000,0.000000
3,0.026341,86.945514,2.278420,0.016929
4,0.015325,208.831053,54.351244,0.307820
...,...,...,...,...
2493,0.015924,130.048217,12.280514,0.163430
2494,0.026830,0.000000,0.000000,0.000000
2495,0.012497,106.206193,10.204455,0.074380
2496,0.004613,146.515532,46.498787,0.040346


In [31]:
type(y)

numpy.ndarray

In [34]:
df['label'] = y.tolist()
df.to_csv('feature-set.csv')

In [39]:
df = pd.read_csv('/home/jovyan/ML/git-code/feature-set.csv').set_index('id') 
df

,0,1,2,3,label
id,,,,,
0,0.025885,249.622739,7.270292,0.018893,False
1,0.010622,154.681365,41.506471,0.087582,False
2,0.033283,0.000000,0.000000,0.000000,True
3,0.026341,86.945514,2.278420,0.016929,False
4,0.015325,208.831053,54.351244,0.307820,True
...,...,...,...,...,...
2493,0.015924,130.048217,12.280514,0.163430,True
2494,0.026830,0.000000,0.000000,0.000000,True
2495,0.012497,106.206193,10.204455,0.074380,False


In [ ]:
X = 

In [25]:
from sklearn import svm
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.3)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((1748, 4), (750, 4), (1748,), (750,))

In [26]:
model  = svm.SVC(C=1)
model.fit(X_train,y_train)

yt_p = model.predict(X_train)
yv_p = model.predict(X_val)

print('Training Accuracy', np.mean(yt_p==y_train))
print('Validation  Accuracy', np.mean(yv_p==y_val))
print('The support vectors are', model.support_vectors_.shape)

Training Accuracy 0.5303203661327232
Validation  Accuracy 0.5226666666666666
The support vectors are (1650, 4)


In [27]:
mean = X_train.mean(0)
sd =  X_train.std(0)

X_train = (X_train-mean)/sd
X_val  = (X_val-mean)/sd

model  = svm.SVC(C=1,gamma=2)
model.fit(X_train,y_train)

yt_p = model.predict(X_train)
yv_p = model.predict(X_val)

print('Training Accuracy', np.mean(yt_p==y_train))
print('Validation  Accuracy', np.mean(yv_p==y_val))
print('The support vectors are', model.support_vectors_.shape)

Training Accuracy 0.6624713958810069
Validation  Accuracy 0.588
The support vectors are (1603, 4)
